举个例子，我们先创建一个人工数据集，然后把它保存为 CSV 文件，再从 CSV 文件中读取数据集。

下面我们将数据集按行写入CSV文件中

In [33]:
import os

In [34]:
os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 写入表头
    f.write('NA,Pave,127500\n')         # 写入第一行数据
    f.write('2,NA,106000\n')            # 写入第二行数据
    f.write('4,NA,178100\n')            # 写入第三行数据
    f.write('3,NA,140000\n')            # 写入第四行数据

要从创建的CSV文件中加载原始数据集，我们导入pandas报，并调用read_csv函数。

该数据集有五行三列，其中每行描述了房间数量、巷道类型和价格。注意，某些数据缺失值用“NA”表示。

In [35]:
import pandas as pd


data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       3.0   NaN  140000


2.2.2处理缺失数据。

处理切实数据的典型方法包括插值和删除缺失数据。插值是用某个合理的值（如均值）替换缺失值。删除缺失数据是直接丢弃包含缺失值的样本或特征。

通过位置索引iloc，我们讲data分成inputs和outputs，其中inputs包含前两列，outputs包含最后一列。

对于inputs中的缺失值，我们用该列的均值进行替换。

In [48]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean())
print(inputs, outputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN 0    127500
1    106000
2    178100
3    140000
Name: Price, dtype: int64


删除有缺失输出数据的样本。

In [38]:
type(inputs) # inputs是一个pandas的DataFrame类型
print(inputs.columns) # 输出DataFrame的列名
inputs_droprows = inputs.dropna(subset=['Alley']) # 删除Alley列中有缺失值的行
inputs_droprows

Index(['NumRooms', 'Alley'], dtype='object')


,NumRooms,Alley
0,3.0,Pave




X对于inputs中的类别值或离散值，我们将NAN视为一个类别。

由于Alley列只接受两种类型的类别之PAVE和NAN，pandas可以自动将此列转换为两列ALLEY_PAVE和ALLEY_NAN。

alley列为pave的行会将ALLEY_PAVE设为1，ALLEY_NAN设为0；alley列为nan的行会将ALLEY_PAVE设为0，ALLEY_NAN设为1。

在 pandas 中，pd.get_dummies(inputs, dummy_na=True) 是用于

对分类变量（categorical variables）进行独热编码（One-Hot Encoding） 的函数，

其核心作用是将非数值型的分类数据转换为机器学习模型可处理的数值型特征。

In [39]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


现在inputs和outputs中的所有条目都是数值类型，

它们可以转换成张量，以便在后续的深度学习模型中使用。

当数据采用张量格式后，可以用2.1节中引入的张量函数操作。

In [40]:
import torch

x, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
x, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

课后小作业：
删除缺失值最多的列

In [42]:
import pandas as pd
import numpy as np

# 构造示例数据（含不同数量的缺失值）
df = pd.DataFrame({
    'A': [1, np.nan, 3, np.nan, 5],  # 2个缺失值
    'B': [np.nan, np.nan, np.nan, 4, 5],  # 3个缺失值（最多）
    'C': [10, 20, np.nan, 40, 50],  # 1个缺失值
    'D': [np.nan, np.nan, np.nan, np.nan, 9],  # 4个缺失值（最多）
    'E': [1, 2, 3, 4, 5]  # 0个缺失值
})
df

,A,B,C,D,E
0,1.0,NaN,10.0,NaN,1
1,NaN,NaN,20.0,NaN,2
2,3.0,NaN,NaN,NaN,3
3,NaN,4.0,40.0,NaN,4
4,5.0,5.0,50.0,9.0,5


In [43]:
missing_per_col = df.isna().sum()
print("每列缺失值数量：")
print(missing_per_col)

每列缺失值数量：
A    2
B    3
C    1
D    4
E    0
dtype: int64


In [44]:
# 2. 找到最大缺失值数量
max_missing = missing_per_col.max()
print("\n最大缺失值数量：", max_missing)


最大缺失值数量： 4


In [45]:
# 3. 筛选出缺失值最多的列（缺失值数量等于最大值的列）
cols_to_drop = missing_per_col[missing_per_col == max_missing].index
print("\n要删除的列：", cols_to_drop)


要删除的列： Index(['D'], dtype='object')


In [46]:
df_clean = df.drop(columns=cols_to_drop)
df_clean

,A,B,C,E
0,1.0,NaN,10.0,1
1,NaN,NaN,20.0,2
2,3.0,NaN,NaN,3
3,NaN,4.0,40.0,4
4,5.0,5.0,50.0,5


如果有多列缺失值数量相同且都是最大值（例如，假设 B 和 D 都有 4 个缺失值），cols_to_drop 会包含这些列，drop() 会一次性删除所有符合条件的列。

若想直接修改原 DataFrame，可添加 inplace=True 参数（不推荐，建议保留原数据）：

df.drop(columns=cols_to_drop, inplace=True)。

该方法适用于需要移除 “信息损失最严重” 的列（缺失值过多的列通常对分析 / 建模意义不大）。

课后作业2：将处理后的数据集转换为张量格式

In [47]:
newdata = df_clean.fillna(df_clean.mean())
newdata = pd.get_dummies(newdata, dummy_na=True)
newdata

,A,B,C,E
0,1.0,4.5,10.0,1
1,3.0,4.5,20.0,2
2,3.0,4.5,30.0,3
3,3.0,4.0,40.0,4
4,5.0,5.0,50.0,5


In [55]:
numpy_array = newdata.values
print(numpy_array)
tensor_data = torch.tensor(numpy_array)
tensor_data

[[ 1.   4.5 10.   1. ]
 [ 3.   4.5 20.   2. ]
 [ 3.   4.5 30.   3. ]
 [ 3.   4.  40.   4. ]
 [ 5.   5.  50.   5. ]]


tensor([[ 1.0000,  4.5000, 10.0000,  1.0000],
        [ 3.0000,  4.5000, 20.0000,  2.0000],
        [ 3.0000,  4.5000, 30.0000,  3.0000],
        [ 3.0000,  4.0000, 40.0000,  4.0000],
        [ 5.0000,  5.0000, 50.0000,  5.0000]], dtype=torch.float64)